In [ ]:
# default_exp uid3

# uid3

> Main module.

In [ ]:
# export
class UId3:
     def __init__(self):
        NODE_SIZE_LIMIT = 1
        TREE_DEPTH_LIMIT = 2
        GROW_CONFIDENCE_THRESHOLD = 0
        

In [ ]:
# export
def main():
    pass

In [ ]:
# export
if __name__ == "__main__":
    main()